In [0]:
from datetime import datetime
def pcounter(current,total,step=100) :
  if current%step == 0 :
    print("Processed {} of {} items".format(current,total))

def Majority(temp2): 
  values=temp2['Party Affiliation'].value_counts(dropna=False).tolist()
  keys=temp2['Party Affiliation'].value_counts(dropna=False).keys().tolist()  
  return (values[0]/temp2.shape[0])*100,keys[0]


def check_divorced(dict_1,dict_2) :
  count_present = 0
  count_absent = 0
  total = len(dict_1)
  count = 0
  for k,v in dict_1.items() :
    
    count += 1
    pcounter(count,total,100000)

    if k not in dict_2.keys() :
      count_absent += 1 
      continue
    else :
      try:
        names_1 = sorted([ v[0]['Name First'].strip() ,v[1]['Name First'].strip() ])
        names_2 = sorted([ dict_2[k][0]['Name First'].strip() , dict_2[k][1]['Name First'].strip()  ])
      except Exception as e:
        pdb.set_trace()

    if names_1 == names_2 :
      count_present += 1
    else :
      count_absent +=1   
  pcounter(count,total,100000)    

  return (count_absent/(count_present+count_absent))*100

def identify_couples(couples) :
  hashmap = {}
  counter = 0
  for index,row in couples.iterrows() :
    counter += 1
    pcounter(counter,couples.shape[0],100000)
    val = row.to_dict()

    val['complete_address'] =  val['complete_address'].replace(' ','')
    
    key = val['complete_address'] + val["Name Last"]

    if key not in hashmap :
      hashmap[key] = []
    hashmap[key].append(val)
  pcounter(counter,couples.shape[0],100000) 
    
  same_party = {}
  diff_party = {}

  total = len(hashmap.keys())
  counter = 0
  for key,value in hashmap.items() :

    if counter%100000 == 0 :
      print("Processed {} of {} items".format(counter,total))

    counter += 1

    if len(value) == 1:
      continue
    if len(value) >= 2 :
      value=sorted(value, key = lambda i: datetime.strptime(i['Birth Date'],"%m/%d/%Y"),reverse=True)
      value[0]['Children']=len(value[2:])
      value[1]['Children']=len(value[2:])
      value=value[0:2]
      d1 = datetime.strptime(value[0]['Birth Date'],"%m/%d/%Y")
      d2 =  datetime.strptime(value[1]['Birth Date'],"%m/%d/%Y")
      diff = abs(relativedelta(d2,d1).years)
      if diff <= 15:
        gen1 = value[0]['Gender'].strip()
        gen2 =  value[1]['Gender'].strip()
        if gen1 == gen2  :
          continue
        value[0]['Party Affiliation'] = value[0]['Party Affiliation'].strip()
        value[1]['Party Affiliation'] = value[1]['Party Affiliation'].strip()
        if value[0]['Party Affiliation'] == value[1]['Party Affiliation'] :
          if key not in same_party :
            same_party[key] = []
            same_party[key] = value 
        else :
          if key not in diff_party :
            diff_party[key] = []      
            diff_party[key] = value


  print("Processed {} of {} items".format(counter,total))
  
  return same_party,diff_party


def couples_by_generation(couples) :

  millenial = [1977,1995]
  boomer = [1946,1964]
  genx = [1965,1976]
  genz = [1996,2019]

  segregated = {
      "millenial" : {},
      "boomer" : {},
      "genz" : {},
      "genx" : {},
      "none" : {}
  }

  for key,couple in couples.items() :
    d1 = datetime.strptime(couple[0]['Birth Date'],"%m/%d/%Y").year
    d2 =  datetime.strptime(couple[1]['Birth Date'],"%m/%d/%Y").year

    if ( millenial[0] <= d1 <= millenial[1] ) and ( millenial[0] <= d2 <= millenial[1] ) :
      segregated["millenial"][key] = couple

    elif ( boomer[0] <= d1 <= boomer[1] ) and ( boomer[0] <= d2 <= boomer[1] ) :
      segregated["boomer"][key] = couple

    elif ( genz[0] <= d1 <= genz[1] ) and ( genz[0] <= d2 <= genz[1] ) :
      segregated["genz"][key] = couple

    elif ( genx[0] <= d1 <= genx[1] ) and ( genx[0] <= d2 <= genx[1] ) :
      segregated["genx"][key] = couple
    
    else :
      segregated["none"][key] = couple

  return segregated

def couples_d(data1,data2):
  dem_dem={}
  rep_rep={}
  rep_dem={}
  dem_rep={}
  others_dem={}
  others_rep={}
  others_others={}
  rep_others={}
  dem_others={}
  for keys,values in data1.items():
    if data1[keys][0]['Party Affiliation'] and data1[keys][1]['Party Affiliation'] == 'DEM':
      dem_dem[keys]= data1[keys]    
    elif data1[keys][0]['Party Affiliation'] and data1[keys][1]['Party Affiliation'] == 'REP':
      rep_rep[keys]=data1[keys]
    elif (data1[keys][0]['Party Affiliation'] and data1[keys][1]['Party Affiliation']) not in('DEM','REP'):
      others_others[keys]=data1[keys]
      

  for keys,values in data2.items():
    if (data2[keys][0]['Party Affiliation'] and data2[keys][1]['Party Affiliation']) in ('DEM','REP'):
      if (data2[keys][0]['Gender'] == 'M' and data2[keys][0]['Party Affiliation'] == 'REP') or (data2[keys][1]['Gender'] == 'M' and data2[keys][1]['Party Affiliation'] == 'REP') :
        rep_dem[keys]=data2[keys]
      elif (data2[keys][0]['Gender'] == 'M' and data2[keys][0]['Party Affiliation'] == 'DEM') or (data2[keys][1]['Gender'] == 'M' and data2[keys][1]['Party Affiliation'] == 'DEM') :
        dem_rep[keys]=data2[keys]
    else:
      if (data2[keys][0]['Gender'] == 'M' and data2[keys][0]['Party Affiliation']=='REP') or (data2[keys][1]['Gender'] == 'M' and data2[keys][1]['Party Affiliation']=='REP'): 
        rep_others[keys]=data2[keys]
      elif (data2[keys][0]['Gender'] == 'M' and data2[keys][0]['Party Affiliation']=='DEM') or (data2[keys][1]['Gender'] == 'M' and data2[keys][1]['Party Affiliation']=='DEM'): 
        dem_others[keys]=data2[keys]
      elif (data2[keys][0]['Gender'] == 'F' and data2[keys][0]['Party Affiliation']=='REP') or (data2[keys][1]['Gender'] == 'F' and data2[keys][1]['Party Affiliation']=='REP'): 
        others_rep[keys]=data2[keys]
      elif (data2[keys][0]['Gender'] == 'F' and data2[keys][0]['Party Affiliation']=='DEM') or (data2[keys][1]['Gender'] == 'F' and data2[keys][1]['Party Affiliation']=='DEM'):
        others_dem[keys]=data2[keys]
        
  return dem_dem,rep_rep,rep_dem,dem_rep,others_others,rep_others,dem_others,others_rep,others_dem


def split_couples_by_age(couples) :

  age_diff = {}
  for i in range(0,16) :
    age_diff[i] = {}

  
  for key,couple in couples.items() :
    d1 = datetime.strptime(couple[0]['Birth Date'],"%m/%d/%Y")
    d2 =  datetime.strptime(couple[1]['Birth Date'],"%m/%d/%Y")
    diff = abs(relativedelta(d2,d1).years)
    if diff <= 15 :
      if diff in age_diff :
        age_diff[diff][key] = couple
  
  return age_diff

###Calculate with/without children groups###
def children(data):
  with_child={}
  without_child={}
  for keys,values in data.items():
    if data[keys][0]['Children'] >= 1:
      with_child[keys]=data[keys]
    else:
      without_child[keys]=data[keys]
  return with_child,without_child


In [0]:

import pandas as pd
import pdb
import os
from google.colab import drive
from datetime import datetime
drive.mount('/content/drive')
import sys
import time
pd.options.display.max_columns = None

#Read all files and merge into 1 ( All counties)
start = time.time()
dataframes = []
counter = 0
total = len(os.listdir('/content/drive/My Drive/FL_2012/')) 
for filename in os.listdir('/content/drive/My Drive/FL_2012/'):
  counter += 1
  if counter%10 == 0 :
    pcounter(counter,total,10)
  if filename.endswith('html') or filename.endswith('tmp') :
    continue
  #try :
  data = pd.read_csv('/content/drive/My Drive/FL_2012/'+filename, sep="\t", header=None,low_memory=False)        
  data.columns = ["County Code","Voter ID","Name Last","Name Suffix","Name First","Name Middle","Requested public records exemption","Residence Address Line 1","Residence Address Line 2","Residence City (USPS)","Residence State","Residence Zipcode","Mailing Address Line 1","Mailing Address Line 2","Mailing Address Line 3","Mailing City","Mailing State","Mailing Zipcode	","Mailing Country","Gender","Race","Birth Date","Registration Date","Party Affiliation","Precinct","Precinct Group","Precinct Split","Precinct Suffix","Voter Status","Congressional District","House District","Senate District","County Commission District","School Board District","Daytime Area Code","Daytime Phone Number","Daytime Phone Extension"] #,"Email address"
  data =  data.loc[:,["Voter ID","Name Last","Name First","Name Middle","Residence Address Line 1","Residence Address Line 2","Residence City (USPS)","Residence State","Residence Zipcode","Gender","Race","Birth Date","Registration Date","Party Affiliation"]]        
  data = data.dropna(subset=["Voter ID", "Name Last","Birth Date","Name First","Residence Address Line 1","Residence Address Line 2","Gender","Party Affiliation"])          
  dataframes.append(data)
  #except Exception as e :
    #print("Failed to read {}".format(filename))
    #continue
pcounter(counter,total,10)
temp_dataframe = pd.DataFrame()
temp_dataframe = pd.concat(dataframes)
temp_dataframe = temp_dataframe[temp_dataframe['Gender'].isin(['M', 'F'])]
temp2 = temp_dataframe
temp2['complete_address'] = temp2['Residence Address Line 1'].str.cat(temp2[['Residence Address Line 2']], sep=' - ')



dataframes = []
counter = 0
total = len(os.listdir('/content/drive/My Drive/FL_2013/')) 
for filename in os.listdir('/content/drive/My Drive/FL_2013/'):
  counter += 1
  if counter%10 == 0 :
    pcounter(counter,total,10)
  if filename.endswith('html') or filename.endswith('tmp') :
    continue
  #try :
  data = pd.read_csv('/content/drive/My Drive/FL_2013/'+filename, sep="\t", header=None,low_memory=False)        
  data.columns = ["County Code","Voter ID","Name Last","Name Suffix","Name First","Name Middle","Requested public records exemption","Residence Address Line 1","Residence Address Line 2","Residence City (USPS)","Residence State","Residence Zipcode","Mailing Address Line 1","Mailing Address Line 2","Mailing Address Line 3","Mailing City","Mailing State","Mailing Zipcode	","Mailing Country","Gender","Race","Birth Date","Registration Date","Party Affiliation","Precinct","Precinct Group","Precinct Split","Precinct Suffix","Voter Status","Congressional District","House District","Senate District","County Commission District","School Board District","Daytime Area Code","Daytime Phone Number","Daytime Phone Extension"] #,"Email address"
  data =  data.loc[:,["Voter ID","Name Last","Name First","Name Middle","Residence Address Line 1","Residence Address Line 2","Residence City (USPS)","Residence State","Residence Zipcode","Gender","Race","Birth Date","Registration Date","Party Affiliation"]]        
  data = data.dropna(subset=["Voter ID", "Name Last","Birth Date","Name First","Residence Address Line 1","Residence Address Line 2","Gender","Party Affiliation"])          
  dataframes.append(data)

  #except Exception as e :
    #print("Failed to read {}".format(filename))
    #continue
pcounter(counter,total,10)



temp_dataframe = pd.DataFrame()
temp_dataframe = pd.concat(dataframes)
temp_dataframe = temp_dataframe[temp_dataframe['Gender'].isin(['M', 'F'])]
temp3 = temp_dataframe
temp3['complete_address'] = temp3['Residence Address Line 1'].str.cat(temp3[['Residence Address Line 2']], sep=' - ')



coupes_same,coupes_diff =  identify_couples(temp2)
coupes1_same,coupes1_diff =  identify_couples(temp3)


print(check_divorced(coupes_same,coupes_same))
print(check_divorced(coupes1_diff,coupes1_diff))
print(time.time() - start)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processed 10 of 67 items
Processed 20 of 67 items
Processed 30 of 67 items
Processed 40 of 67 items
Processed 50 of 67 items
Processed 60 of 67 items
Processed 10 of 67 items
Processed 20 of 67 items
Processed 30 of 67 items
Processed 40 of 67 items
Processed 50 of 67 items
Processed 60 of 67 items
Processed 100000 of 12175065 items
Processed 200000 of 12175065 items
Processed 300000 of 12175065 items
Processed 400000 of 12175065 items
Processed 500000 of 12175065 items
Processed 600000 of 12175065 items
Processed 700000 of 12175065 items
Processed 800000 of 12175065 items
Processed 900000 of 12175065 items
Processed 1000000 of 12175065 items
Processed 1100000 of 12175065 items
Processed 1200000 of 12175065 items
Processed 1300000 of 12175065 items
Processed 1400000 of 12175065 items
Processed 1500000 of 12175065 items
Processed 1600000 of 12175065 items
Proc

In [0]:

dem_dem,rep_rep,rep_dem,dem_rep,others_others,rep_others,dem_others,others_rep,others_dem = couples_d(coupes2016_same,coupes2016_diff)
with_child,without_child=children(coupes2016_same)
with_child_17,without_child_17=children(coupes2017_same)

diff_with,diff_without=children(coupes2016_diff)
diff_with_2017,diff_without_2017=children(coupes2017_diff)

print(len(list(with_child.values())))
print(len(list(without_child.values())))
print(len(list(with_child_17.values())))
print(len(list(without_child_17.values())))

print(list(with_child.values())[2])
print(list(without_child.values())[2])

print(check_divorced(with_child,with_child_17))
print(check_divorced(without_child,without_child_17))

print(check_divorced(diff_with,diff_with_2017))
print(check_divorced(diff_without,diff_without_2017))








1139
26837
1157
27124
[{'Voter ID': 119253110, 'Name Last': 'Carney', 'Name First': 'Kristen', 'Name Middle': 'Ilean', 'Residence Address Line 1': '6223  NW 41st Dr ', 'Residence Address Line 2': ' ', 'Residence City (USPS)': 'Gainesville', 'Residence State': ' ', 'Residence Zipcode': '32653', 'Gender': 'F', 'Race': 5, 'Birth Date': '01/13/1995', 'Registration Date': '11/18/2011', 'Party Affiliation': 'DEM', 'complete_address': '6223NW41stDr-', 'Children': 2}, {'Voter ID': 115177855, 'Name Last': 'Carney', 'Name First': 'Robert', 'Name Middle': 'Marion', 'Residence Address Line 1': '6223  NW 41st Dr ', 'Residence Address Line 2': ' ', 'Residence City (USPS)': 'Gainesville', 'Residence State': ' ', 'Residence Zipcode': '32653', 'Gender': 'M', 'Race': 5, 'Birth Date': '10/29/1988', 'Registration Date': '05/07/2007', 'Party Affiliation': 'DEM', 'complete_address': '6223NW41stDr-', 'Children': 2}]
[{'Voter ID': 100022686, 'Name Last': 'Tracy', 'Name First': 'Dennis', 'Name Middle': 'J', 'R

In [0]:
k=list(coupes2016_same.values())[0]
#print(list(coupes2016_same.values())[1])
# (data2[keys][0]['Party Affiliation'] not in ('DEM','REP')) or (data2[keys][1]['Party Affiliation'] not in ('DEM','REP')):
#print(k[0]['Party Affiliation'])
#print(list(coupes2016_same.keys())[0])
#print(coupes2016_same['7948NW49ThWay-Haskins'][1]['Party Affiliation'])
#print(coupes2016_same['13505NW167ThTer-Daughtry'])
#print(coupes2016_same['132NW117ThWay-Dryfuss'])
print(len(list(others_others.values())))
print(len(list(others_dem.values())))
print(len(list(others_rep.values())))
print(len(list(dem_others.values())))
print(len(list(rep_others.values())))
#print(list(dem_dem.values())[1])
age_dd=split_couples_by_age(coupes2016_same)
#print(age_dd[1]['7948NW49ThWay-Haskins'])
print(list(with_child.values())[2])
print(list(without_child.values())[2])
